In [1]:
def load_fasta(filepath):
    """
    Load a FASTA file and return the DNA sequence as a single continuous string.
    Header lines starting with '>' are skipped.
    """
    sequence = []
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if not line.startswith(">"):
                sequence.append(line)
    return "".join(sequence)

In [2]:
def make_cDNA(genome):
    """
    Extract exons from genomic DNA (HBB gene) and concatenate to create cDNA.
    Exon coordinates are based on NG_000007.3 reference.
    NOTE: Coordinates below are relative to our HBB_region.fasta slice (offset 70545).
    In other words, the FASTA file we have is just a slice of the full genome, starting at position 70,545.
    """

    offset = 70545 # Genomic start position of the HBB_region.fasta slice (example)

    # The genomic coordinates on NG_000007.3 are converted to local slice coordinates by substracting the defined offset.
    exons = [
        (70573 - offset, 70714 - offset), # exon 1: convert the global genomic coordinates into the local slice coordinates
        (70955 - offset, 71083 - offset), # exon 2: convert the global genomic coordinates into the local slice coordinates
        (71144 - offset, 71261 - offset), # exon 3: convert the global genomic coordinates into the local slice coordinates
    ]

    print("=== Extracting Exons ===")
    cDNA = ""
    for idx, (start, end) in enumerate(exons, 1):
        exon_seq = genome[start:end]
        print(f"Exon {idx}: positions {start} ~ {end}")
        print(f"Sequence ({len(exon_seq)} bp): {exon_seq[:50]}...\n")  # print firtst 50 sequence
        cDNA += exon_seq

    print("=== Done ===")
    print(f"Final cDNA length: {len(cDNA)} bp")

    return cDNA

#-------------------------

def trim_from_first_ATG(cDNA):
    """
    Return coding sequence starting from the first ATG in ther cDNA.
    If no ATG is found, return the original cDNA.
    """
    start_index = cDNA.find("ATG")
    if start_index != -1:
        return cDNA[start_index:]
    return cDNA

In [ ]:
class TSTNode:
    def __init__(self, _character):
        self.character = _character
        self.val = None
        self.left = None
        self.right = None
        self.middle = None
    
    def __repr__(self):
        if self.val:
            return f"{self.character} ({self.val})"
        return self.character


class TST:
    def __init__(self):
        self.root = None

    def put(self, sequence, val):
        self.root = self._put(self.root, sequence, val, 0)

    def _put(self, current_node, sequence, val, index):
        character = sequence[index]
        if current_node is None:
            current_node = TSTNode(character)
        
        if current_node.character == character:
            if index == len(sequence) - 1: # last char in sequence
                ########## Changed here
                if current_node.val is None:
                    # make a list if the key is the first key being put
                    current_node.val = [val]
                else:
                    # if the list exists, just add to the list
                    current_node.val.append(val)
                ##########
            else:
                current_node.middle = self._put(current_node.middle, sequence, val, index + 1)
        elif character < current_node.character:
            current_node.left = self._put(current_node.left, sequence, val, index)
        else:
            current_node.right = self._put(current_node.right, sequence, val, index)

        return current_node
    
    def contains(self, sequence):
        return self.get(sequence) is not None
    
    def get(self, sequence):
        node = self._get(self.root, sequence, 0)
        if node is None:
            return None
        return node.val
    
    def _get(self, current_node, sequence, index):
        if current_node is None:
            return None
        character = sequence[index]
        if current_node.character == character:
            if index == len(sequence) - 1: # last char in sequence
                return current_node
            else:
                return self._get(current_node.middle, sequence, index + 1)
        elif character < current_node.character:
            return self._get(current_node.left, sequence, index)
        else:
            return self._get(current_node.right, sequence, index)

In [4]:
### Sanity Check Using a simple example (TST Version)

# Create TST instance
tst = TST()

# Example DNA sequence split into overlapping substrings of length M
genome = "ACGTACGTAA"
pattern = "CGT"
M = len(pattern)

# Insert all substrings of length M into the TST
for i in range(len(genome) - M + 1):
    substring = genome[i:i+M]
    tst.put(substring, i)   # Store starting index as value

# Search for pattern in the TST
result = tst.get(pattern)

print("Genome:  ", genome)
print("Pattern: ", pattern)

if result is None:
    print("Match not found")
else:
    print("Match found at index:", result)

Genome:   ACGTACGTAA
Pattern:  CGT
Match found at index: [1, 5]


In [6]:
if __name__ == "__main__":
    genome = load_fasta("HBB_region.fasta")
    cDNA = make_cDNA(genome)
    coding_seq = trim_from_first_ATG(cDNA)
    pattern = "GTG" # sickle-cell mutation codon

    tst= TST()

    # Enter every 3-mer codon into the TST (but in this case it is only "GTG")
    for i in range(len(coding_seq) - 2):
        codon = coding_seq[i:i+3]
        tst.put(codon, i)

    # Find the location of pattern
    if tst.contains(pattern):
        print("GTG found at position:", tst.get(pattern))
    else:
        print("GTG not found")

=== Extracting Exons ===
Exon 1: positions 28 ~ 169
Sequence (141 bp): TGTAACAGAATAAAAAATCAATTATGTATTCAAGTTGCTAGTGTCTTAAG...

Exon 2: positions 410 ~ 538
Sequence (128 bp): CAAGGGATGGATGAAGGCAGGTGACTCTAACAGAAAGGGAAAGGATGTTG...

Exon 3: positions 599 ~ 716
Sequence (117 bp): ACTTTGAGTTTGTAAGTATATACTTCTCTGTAATGTGTCTGAATATCTCT...

=== Done ===
Final cDNA length: 386 bp
GTG found at position: [17, 138, 280, 328]


# Analysis of the Sickle Cell Data
After extracting the exons from the HBB_region.fasta and joining them together, we converted the sequence into a continuous cDNA starting from the first ATG—the translation start site of β-globin. Because the coordinate system is now “protein-coding only”, every nucleotide position in the output corresponds directly to codons in the mRNA, not the original genomic positions. After generating the final 386-bp cDNA, we searched for the “GTG” codon. Several GTG sites showed up, which is normal because valine occurs multiple times throughout the β-globin protein.

The reason the GTG located around position 17 is important is that this is the codon corresponding to the classical sickle-cell mutation. In the beta-globin gene, sickle-cell disease is caused by a single A to T substitution that converts the sixth amino-acid codon from GAG (Glu) to GTG (Val). Since each codon is three nucleotides, the sixth codon starts around nucleotide positions 16–18 in the cDNA. That is why the code identifies a GTG at the position 17: among all GTG hits, this specific one represents the disease-causing mutation hotspot.

## Time Complexity — O(P × M)

We store patterns in a Ternary Search Tree (TST).
Each pattern is searched individually in the pre-built TST.

A TST lookup takes O(M) time because we follow one character per level through
the middle pointers, and each character comparison (<, =, >) is O(1).

For P patterns, the total runtime is:

`O(P × M)`

where P is the number of patterns and M is the pattern length.

Although the worst-case complexity matches brute-force search for single patterns, the TST avoids
fully restarting the comparison after each mismatch, making it faster in
practice for long genomic sequences, especially when searching multiple patterns.

## Space Complexity — O(M)

The TST stores only the pattern. Each character becomes one node with three
pointers (left, middle, right). Therefore the structure grows linearly with the
pattern length:

`O(M)`

Search requires no additional memory proportional to the genome length.



# Why TST Is Still Useful for Codon Search

Even though the time-complexity expression for a TST looks a bit heavy at first, it does not mean the method is worse than brute-force. In practice, a TST becomes faster because it does not restart the entire comparison at every position in the genome. Instead, it reuses its branching structure to guide each search.  

This helps especially when we need to search for multiple codons or motifs. Brute-force has to scan through the whole sequence again for every single pattern, but a TST uses the same tree for all queries. For P patterns, TST has `O(P × M)` time complexity, which is much more efficient than brute-force's `O(P × N × M)`.

Because of this, a TST is a good fit for biological sequences where we often look for several codons or variants at once. After the tree is built, each additional lookup becomes very fast compared to scanning the whole genome again.

# Applying It to the E. coli **lacZ** Gene

To show this behavior more clearly, we apply our TST implementation to the *E. coli* **lacZ** gene. The goal is to search for multiple GTG codons (and potentially other patterns) inside a real gene, not just a small toy example.  

Using a real sequence like **lacZ** highlights when the TST starts to outperform simple brute-force:  
- we only build the tree once,  
- then we reuse it for each codon we want to test,  
- and we avoid re-scanning the entire sequence every time.

This makes the comparison more realistic and shows why TST becomes valuable when dealing with larger genomes or repeated pattern searches in bioinformatics.

## Applying TST to the *E. coli* lacZ Gene

When we switch from small toy examples to a real gene like **lacZ**, the benefit of TST becomes clearer.  
The key idea is that TST lets us handle multiple codons** much more efficiently than brute-force.

### Why TST helps here
- We only build the TST once using all codons (or any patterns we want to search).
- Each pattern is searched individually in the pre-built TST.
- For each pattern, we traverse the TST following the pattern characters.

Because the TST lookup is efficient, requiring exactly M comparisons per pattern, the overall search time for P patterns is
`O(P × M)` time complexity — where P is the number of patterns and M is the pattern length.

This is much cheaper than brute-force, which is
`O(P × N × M)` time complexity — where P is the number of patterns, N is the sequence length, and M is the full length of each pattern, since brute-force must rescan the entire gene for every pattern separately.

### Why this matters
This means TST becomes useful when:
- we test many codons (not just a single codon)
- or run repeated scans over a long gene like lacZ,
- without paying the cost of full re-scans every time.

So even though the speedup is small when searching for only one pattern,
lacZ is where the advantage of handling many patterns at once finally becomes noticeable.